# Relational Databases: ORM - Using `SQLAlchemy` with Python

`SQLAlchemy` is an example for a ORM (Object Relational Mapper). This is a high-level interface to interact with DBs of all common types. Another concept/interface is a DB-API. An example is shown in the [db-sqlite3-notebook](db-sqlite3.ipynb).
Tabels and queries are mapped to Python objects and functions. An ORM uses a DB-API under the hood, but the user does not need to know and write SQL queries. It is a more "pythonic" way to interact with a DB.

**Use**, if you have:
- Complex database shemas and larg projects
- Evolving DB including version control and migration
- The need to plug the same code to different RDBMS (Relational Database Management Systems)

**Don't use**, if you have:
- Only one static DB that has to populated and queried once
- (You want to avoid the overhead of defining the table classes)
- The need for fine-grained control over all DB connections (usually not the case!)
- Extremely good SQL skills and no understanding of Python at all

In [ ]:
import pandas as pd
from sqlalchemy import create_engine, UniqueConstraint, select
from sqlalchemy.orm import DeclarativeBase
from typing import List, Set
from typing import Optional
from sqlalchemy.orm import Mapped, mapped_column, relationship, sessionmaker
from sqlalchemy import String, ForeignKey
from datetime import datetime

In [ ]:
import sqlalchemy
sqlalchemy.__version__  

# Load test data

For this demonstration, we load data from .csv-files into Pandas dataframes and put them into our sqlite database afterwards.
We do this mostly to use the nice dataframe visualization.
In general, it is not necessary to create dataframes before inserting data into a DB!

In [ ]:
# Load Teams table
df_teams = pd.read_csv(
    "data/teams.csv",
    sep=";",
    index_col="ID",
    usecols=["ID", "Size", "Shoe Color"]
)
# Load Runner table
df_runners = pd.read_csv(
    "data/participants.csv",
    sep=";",
    usecols=["First Name", "Last Name", "Shoe Size", "Shirt Size", "Distance", "Team"]
)
# Load Trainings table
df_training = pd.read_csv(
    "data/training.csv",
    sep=";",
    usecols=["Date(YYYY-MM-DD)", "Time(mm:ss)", "Distance(km)", "Runner"]
)

In [ ]:
for df in [df_runners, df_teams, df_training]:
    display(df)

# Defining the DB Scheme

We want to create a DB containing the tables `teams`, `runners` and `trainings` like:

![RDB_example](RDB_example.png)

A DB has much more functionalty than a simple .csv or Excel table.
This comes with some downsides.
We cannot simply insert data into an empty table, because there is no table at first.
First, we have to specify how our table is designed.
This means we have to define all columns, the data types and the relationships between different tables up front.

(*Yes*, you can avoid this for simple and qick-and-dirty cases by using some `pandas`/`sqlalchemy` magic, but for more complex projects and real use cases, you probably can't overcome it 🤷‍♀️)

In `sqlalchemy`, each table is a Python `class`, which inherits from `sqlalchemy`'s `DeclarativeBase`.
Within each table class, the columns and their characteristics (data types, relationships, primary keys, nullable, ...) are defined.

Some specialities up front:
* Python classes have CamelCase names by convention. Therefore, the table class is named like `Team` (singular!). The SQL name of the table is defined in the `__tablename__` attribute and is usually a lowercase string and plural, like `"teams"`
* Following the new (as of 2023) `sqlalchemy 2.0` guidelines, a column is defined like:
    * `<column_name>: Mapped[<data_type>] = mapped_column(<options>)`
* `Mapped`
    * The Mapped annotation takes care of the correct translation between Python and RDBMS data types
    * If a column is defined like `Mapped[Optional[float]]`, the fiel is allowed to be empty
* `UniqueConstraint` 
    * A UniqueConstraint is used to tell the DB that for specific columns or combinations of columns the values must be unique and can not be inserted a second time. For example, we want every runner only one time in our table of runners.
    * If a UniqueConstraint failed as we try to insert duplicates into our table, we will see an `IntegrityError`
* `relationship` 
    * Relationships are introduced by the relationship() function and a mapped_column to the primary key column of the related table
* `__repr__` 
    * This function defines what is printed if we just print table row (which is an instance of a table class). Defining it is nice to have but not crucially necessary.
 

More information can be found here:
* Sqlalchemy Quick start tutorial: https://docs.sqlalchemy.org/en/20/orm/quickstart.html
* Declarative table configuration: https://docs.sqlalchemy.org/en/20/orm/declarative_tables.html#orm-declarative-table

In [ ]:
class Base(DeclarativeBase):
    pass

In [ ]:
# class name CamelCase, __tablename__ lowercase!
class Team(Base):
    __tablename__ = "teams"
    __table_args__ = (
        # We want each team to have a different shoe color, so we use a UniqueConstraint.
        UniqueConstraint("shoe_color"),
    )
    # The id column is always our primary key
    id: Mapped[int] = mapped_column(primary_key=True)
    size: Mapped[int]
    # We define a maximum length or 20 for the string containing the shoe color 
    shoe_color: Mapped[str] = mapped_column(String(20))
    # A team can have many members, so we establish a one-to-many relationship to the 'team' column in the 'Runner' table.
    members: Mapped[List["Runner"]] = relationship(back_populates="team")
    def __repr__(self) -> str:
        return f"Team {self.id!r} wears {self.shoe_color!r} shoes and has {self.size!r} members."


class Runner(Base):
    __tablename__ = "runners"
    __table_args__ = (
        UniqueConstraint("first_name", "last_name", "team_id"),
    )
    id: Mapped[int] = mapped_column(primary_key=True)
    first_name: Mapped[str] = mapped_column(String(30), nullable=False)
    last_name: Mapped[str] = mapped_column(String(30), nullable=False)
    shoe_size: Mapped[int]
    shirt_size: Mapped[int]
    # We want to allow runners to leave the 'distance' column empty, so we specify the data type as 'Optional[float]'
    distance: Mapped[Optional[float]]
    # The team_id should be a foreign key pointing to the 'id' column of the 'Team' table
    team_id = mapped_column(ForeignKey("teams.id"))
    # Here, we establish the many-to-one relationship to the 'members' column of the 'Team' table 
    team: Mapped[Team] = relationship(back_populates="members")
    # A runner can have many training runs, so we create a one-to-many relationship to the 'runner' column of the 'Training' table
    trainings: Mapped[List["Training"]] = relationship(back_populates="runner")
    def __repr__(self) -> str:
        return f"Runner(id={self.id!r}, first_name={self.first_name!r} last_name={self.last_name!r})"


class Training(Base):
    __tablename__ = "trainings"
    __table_args__ = (
        UniqueConstraint("runner_id", "date"),
    )
    id: Mapped[int] = mapped_column(primary_key=True)
    date: Mapped[datetime]
    time: Mapped[datetime]
    distance: Mapped[float]
    runner_id = mapped_column(ForeignKey("runners.id"))
    runner: Mapped[Runner] = relationship(back_populates="trainings")
    def __repr__(self) -> str:
        return f"Training(id={self.id!r}, runner={self.runner.first_name!r} {self.runner.last_name!r} time={self.time!r} dictance={self.distance!r})"

# Building a DB

## Creating an engine

Since a DB is not necessarily a single file on your disk, we have to define, where our DB lives and how we can connect to it.
Therefore, we build an `Engine` which can handle connections.

The engine gets a connetion string, which contains the type of the RDBMS (e.g. `sqlite` or `postgresql`), a dialect (depends on the RDBMS we use), a location, and maybe user credentials.
For our small example, we work with an sqlite DB here, that lives in memory an is not persisted on disk.
The `echo=True` only switches on some logging, which we can use to understand how the engine works. 

In [ ]:
engine = create_engine("sqlite+pysqlite:///firmenlauf_demo_sqlalchemy.db", echo=True)

## Creating all tables

After we have carefully designed our tables in Python, we now want them to be actually created via SQL statements in our sqlite DB we connected our engine to.

In [ ]:
# open a connection and create all tables that inherit from Base
with engine.begin() as conn:
    Base.metadata.create_all(conn)

# Since we have 'echo=True' specified in our engine, sqlalchemy will tell us what it does

## Fill the tables

For every interaction with a database, we have to create a session. The overall workflow is:
* `begin` a session
* do something, for example `add` a row
* `commit` the changes to the DB
* `rollback` the session, if anything went wrong
* `close` the session

If something went wrong, it can happen that we blow up the connection to the DB.
No problem in our small test case, but in real production this can cause severe loss of data!
Therefore you always have to take care of correctly handled sessions!

We will now loop over our Pandas dataframes and add every row to the corresponding DB table.

**If you run the following "Fill" cells twice, you will receive an `IntegrityError`!**

This is caused to the `UniqueConstraint` we set for all tables. If we try to insert duplicates into the DB, we try to corrupt the intergity of our DB and an error is raised. In real world applications, you always have to deal with this kind of errors via context manager or try/except blocks.

In [ ]:
# We use sqlqlachmy's sessionmaker to handle the sessions
Session = sessionmaker(engine)

### Fill Team table

In [ ]:
# This context manager opens a session and runs a session.commit() as we leave it and session.rollback() in case of exceptions
with Session.begin() as session:
    for id, (size, shoe_color) in df_teams.iterrows():
        team = Team(
            id=id,
            size=size,
            shoe_color=shoe_color
        )
        session.add(team)
    #context manager runs session.commit() and session.close() here


### Fill Runner table

In [ ]:
with Session.begin() as session:
    for id, (first_name, last_name, shoe_size, shirt_size, distance, team_id) in df_runners.iterrows():
        print(first_name, last_name)
        runner = Runner(
            first_name=first_name,
            last_name=last_name,
            shoe_size=shoe_size,
            shirt_size=shirt_size,
            distance=distance,
            team_id=team_id            
        )
        session.add(runner)

### Fill Training table

In [ ]:
with Session.begin() as session:
    for id, (date, time, distance, runner_id) in df_training.iterrows():
        training = Training(
            date=datetime.strptime(date, "%Y-%m-%d"),
            time=datetime.strptime(time, "%M:%S"),
            distance=distance,
            runner_id=runner_id,
        )
        session.add(training)

# Selecting data from the DB

sqlalchemy makes the DB accessible for us, even if we don't speak SQL.
Instead, we speak Python functions and sqlalchemy translates it into SQL queries which are sent to the DB.
The result is then again translated by sqlalchemy into Python objects for us to use.

Again, for every interaction with the DB, we have to open a session, respectively work in a context manager.

## `SELECT * FROM teams`
"Give me a list of all teams!"


In [ ]:
with Session.begin() as session:
    teams = session.execute(
        select(Team)
    ).all()
    print("----- Query Result -----")
    print(teams)
    print("------------------------")

## `SELECT * FROM teams WHERE size > 15;`
"Give me a list of all teams with more than 15 members!"

In [ ]:
with Session.begin() as session:
    teams = session.execute(
        select(Team)
        .where(Team.size > 15)
    ).all()
    print("----- Query Result -----")
    print(teams)
    print("------------------------")

## `SELECT shoe_color FROM teams WHERE size > 15;`
"Give me a list of the shoe colors of all teams with more than 15 members!"

In [ ]:
with Session.begin() as session:
    shoe_colors = session.execute(
        select(Team.shoe_color)
        .where(Team.size > 15)
    ).all()
    print("----- Query Result -----")
    print(shoe_colors)
    print("------------------------")

## `JOIN`

`SELECT first_name, shoe_size, shoe_color  
FROM runners 
 JOIN teams ON runners.team = teams.id;`

"Give me a list of participants including their shoe size and team shoe color!"

We have two options here to get our results:
* run the table `JOIN` explicitly
    * we use the `.join()` function to join the tables as we query it
    * more code, but more controll over queries
* use the `relationship` we have established by defining our table classes!
    * less code, more pythonic
    * maybe less performant or more (sub-)queries under the hood

In [ ]:
# Use JOIN
# We end up with only one query including a JOIN statement!
with Session.begin() as session:
    runners = session.execute(
        select(Runner.first_name, Runner.shoe_size, Team.shoe_color)
        .select_from(Runner)
        .join(Team)
    ).all()
    print("----- Query Result -----")
    print(runners)
    print("------------------------")

In [ ]:
# Use the relationship
with Session.begin() as session:
    runners = session.execute(
        select(Runner)
    ).scalars().all()
    for runner in runners:
        print("----- Query Result -----")
        print(runner.first_name, runner.shoe_size, runner.team.shoe_color)
        print("------------------------")

## Use relations
Another example of the `sqlalchemy` magic would be the following query...

Note, that sqlalchemy sends a new query to the database in the moment we call `team.members`! This is why the session is still needed. We can specify this behavior in the relationship definition and load the depending object directly ("Eager Loading").

See https://docs.sqlalchemy.org/en/20/orm/queryguide/relationships.html for more details!

In [ ]:
with Session.begin() as session:
    teams = session.execute(
        select(Team)
    ).scalars().all()
    
    for team in teams:
        print("----- Query Result -----")
        print(team.members)
        print("------------------------")

# Further Reading

* Relationship patterns: https://docs.sqlalchemy.org/en/20/orm/basic_relationships.html
* engine definition for different RDBMSs: https://docs.sqlalchemy.org/en/20/core/engines.html#database-urls
* Working with ORM related objects: https://docs.sqlalchemy.org/en/20/tutorial/orm_related_objects.html
* All you ever want to know: https://docs.sqlalchemy.org/en/20/tutorial/index.html#unified-tutorial

---
_This notebook is licensed under a [Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International (CC BY-NC-SA 4.0)](https://creativecommons.org/licenses/by-nc-sa/4.0/). Copyright © [Point 8 GmbH](https://point-8.de)_